In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
random.seed(0)

In [3]:
iris = load_iris()

In [4]:
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['species'] = iris.target

In [8]:
type(iris.data)

numpy.ndarray

In [10]:
type(iris.target)

numpy.ndarray

In [6]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
species              150 non-null int64
dtypes: float64(4), int64(1)
memory usage: 5.9 KB


In [11]:
X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                    iris.target,
                                                    test_size=0.2,
                                                    stratify=iris.target)

In [12]:
def check_purity(y):
    unique_classes = np.unique(y)
    if len(unique_classes)==1:
        return True
    else:
        return False

In [13]:
def classify_data(y):
    unique_classes, counts = np.unique(y, return_counts=True)
    return unique_classes[np.argmax(counts)]

In [14]:
def get_potential_splits(X):
    _,n = X.shape
    potential_splits = {}
    for i in range(n):
        # print i
        unique_values = np.unique(X[:,i])
        splits = []
        # print unique_values
        for j in range(1,len(unique_values)):
            splits.append((unique_values[j]+unique_values[j-1])/2.0)
        # print splits
        potential_splits[i] = splits
    return potential_splits

In [15]:
potential_splits = get_potential_splits(X_train)

In [16]:
def split_data(X, y, split_column, threshold):
    below_rows = np.where(X[:,split_column]<=threshold)[0]
    # print below_rows
    above_rows = range(X.shape[0])
    above_rows = np.setdiff1d(above_rows, below_rows)
    # print above_rows
    X_below = X[below_rows,:]
    y_below = y[below_rows]
    X_above = X[above_rows,:]
    y_above = y[above_rows]
    return X_below, X_above, y_below, y_above

In [17]:
def split_y(X, y, split_column, threshold):
    below_rows = np.where(X[:,split_column]<=threshold)[0]
    above_rows = range(X.shape[0])
    above_rows = np.setdiff1d(above_rows, below_rows)
    y_below = y[below_rows]
    y_above = y[above_rows]
    return y_below, y_above

In [18]:
X_below, X_above, y_below, y_above = split_data(X_train, y_train, 0, 5.2)

In [19]:
def calculate_entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts*1.0 / np.sum(counts)
    entropy = -1*(np.sum(probabilities*np.log2(probabilities)))
    return entropy

In [20]:
print calculate_entropy(y_above)

1.3600034483643832


In [21]:
def calculate_overall_entropy(y_below, y_above):
    n_points = len(y_below) + len(y_above)
    overall_entropy = (len(y_below)*calculate_entropy(y_below) + len(y_above)*calculate_entropy(y_above))/n_points
    return overall_entropy

In [22]:
calculate_overall_entropy(y_below, y_above)

1.1469171728542913

In [23]:
def determine_best_split(X, y):
    potential_splits = get_potential_splits(X)
    split_column = None
    split_threshold = None
    min_overall_entropy = 999
    for column, splits in potential_splits.iteritems():
        for threshold in splits:
            y_below, y_above = split_y(X, y, column, threshold)
            overall_entropy = calculate_overall_entropy(y_below, y_above)
            if overall_entropy < min_overall_entropy:
                split_column = column
                split_threshold = threshold
                min_overall_entropy = overall_entropy
    return split_column, split_threshold

In [24]:
determine_best_split(X_train, y_train)

(2, 2.5999999999999996)

In [26]:
class DecisionTreeClassifier:
    def __init__(self, min_samples = 10, max_depth = 3):
        self.is_leaf = False
        self.leaf_value = None
        self.below = None
        self.above = None
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.split_column = None
        self.split_threshold = None
    def fit(self, X, y):
        if len(y)<=self.min_samples or check_purity(y) or self.max_depth == 0:
            self.is_leaf = True
            self.leaf_value = classify_data(y)
        else:
            split_column, split_threshold = determine_best_split(X, y)
            self.split_column = split_column
            self.split_threshold = split_threshold
            X_below, X_above, y_below, y_above = split_data(X, y, split_column, split_threshold)
            self.below = DecisionTreeClassifier(self.min_samples, self.max_depth - 1)
            self.below.fit(X_below, y_below)
            self.above = DecisionTreeClassifier(self.min_samples, self.max_depth - 1)
            self.above.fit(X_above, y_above)
    def predict(self, x):
        if self.is_leaf:
            return self.leaf_value
        else:
            if x[self.split_column] <= self.split_threshold:
                return self.below.predict(x)
            else:
                return self.above.predict(x)

In [27]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [32]:
predictions = []
for row in X_test:
    predictions.append(clf.predict(row))
(predictions == y_test)

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [31]:
X_train

array([[5. , 3.4, 1.6, 0.4],
       [6.1, 2.6, 5.6, 1.4],
       [6.3, 3.4, 5.6, 2.4],
       [4.8, 3.4, 1.6, 0.2],
       [6.6, 3. , 4.4, 1.4],
       [6.7, 2.5, 5.8, 1.8],
       [4.6, 3.6, 1. , 0.2],
       [4.4, 3.2, 1.3, 0.2],
       [5.4, 3. , 4.5, 1.5],
       [6.7, 3. , 5.2, 2.3],
       [7.4, 2.8, 6.1, 1.9],
       [6.5, 3. , 5.2, 2. ],
       [6. , 3.4, 4.5, 1.6],
       [5.1, 3.5, 1.4, 0.2],
       [7.2, 3. , 5.8, 1.6],
       [6.4, 2.7, 5.3, 1.9],
       [5.7, 2.8, 4.1, 1.3],
       [5.4, 3.9, 1.7, 0.4],
       [4.9, 3.1, 1.5, 0.1],
       [5. , 2. , 3.5, 1. ],
       [5.7, 2.8, 4.5, 1.3],
       [6.2, 3.4, 5.4, 2.3],
       [5.8, 2.8, 5.1, 2.4],
       [4.6, 3.1, 1.5, 0.2],
       [7.6, 3. , 6.6, 2.1],
       [5.7, 3. , 4.2, 1.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.1, 2.9, 4.7, 1.4],
       [5.7, 2.9, 4.2, 1.3],
       [6.7, 3.3, 5.7, 2.1],
       [6.4, 2.9, 4.3, 1.3],
       [5.2, 4.1, 1.5, 0.1],
       [5.1, 3.8, 1.5, 0.3],
       [6.2, 2.2, 4.5, 1.5],
       [6. , 2

In [33]:
y_train

array([0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 1, 0, 2, 2, 1, 0, 0, 1, 1, 2,
       2, 0, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 1, 1, 0, 0, 1, 2, 0, 0, 0,
       0, 2, 1, 1, 0, 2, 2, 2, 2, 0, 1, 2, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0,
       1, 2, 1, 1, 1, 2, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 2, 0, 0, 1, 1, 2,
       2, 0, 1, 2, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 1, 0, 0, 2, 1, 2, 2, 1,
       0, 0, 2, 2, 2, 1, 0, 2, 1, 0])